In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write

In [ ]:
import copy # deepcopy
import ipywidgets # widget.Text
import IPython.display # DisplayHandle

In [ ]:
# TODO change to a general display widget on button click
class WidgetPrint(VBox):
    """ Interactive function widget
    
    Creates a standard plot widget that takes a plotting function, and a parameter widget
    and returns an interactive plot generated by calling the plotter with the specified parameters. 
                
    Parameters
    ----------
    plotter: function    
        A function that takes an Axes object, and a list of named parameters, and populates the ax
        with a drawing generated based on the parameters
        
    parbox: WidgetParbox
        A WidgetParbox widget containing the parameters for manipulating the plot
    
    fixed_args: dict
        Parameters that are passed to plotter() without being associated to an interactive widgtet        
    """
    
    def __init__(self, widget, parbox):
        self._parbox = parbox
        self._widget = widget
        self._plot = Output()
        cell_length
        super(WidgetPrint, self).__init__([self._parbox, self._plot])
        self._parbox.observe(self.update)
            
    def update(self):
        display(self._widget)

In [ ]:
class WidgetChemiscope(VBox):
    """ Interactive chemiscope widget
    
                
    Parameters
    ----------
    plotter: chemiscope.jupyter.StructureWidget    
        
    parbox: WidgetParbox
        A WidgetParbox widget containing the parameters for manipulating the plot
    
    fixed_args: dict
        Parameters that are passed to plotter() without being associated to an interactive widgtet        
    """
    
    def __init__(self, widget_code_input, parbox, fixed_args={}):
        
        self._widget_code_input = widget_code_input
        self._fixed_args = fixed_args
        self._parbox = parbox
        self._plot = Output()
        self._handle = IPython.display.DisplayHandle()
        self._cs = None
        
        super(WidgetChemiscope, self).__init__([self._parbox, self._plot])
        self._parbox.observe(self.update)
            
    def update(self):
        structure = self._widget_code_input.get_function_object()(**self._parbox.value, **copy.deepcopy(self._fixed_args))
        
        #TODO add properties when cutoff is fixed
        properties = {}
        properties["index"] = {
                        "target": "atom",
                        "values": list(range(1, len(structure)+1)),
                    }

        properties["coordinates"] = {
                        "target": "atom",
                        "values": list(map(lambda x: str(x), structure.positions.tolist())),
                    }
        #TODO fractional coordinates
        properties["fractional coordinates"] = {
                        "target": "atom",
                        "values": list(map(lambda x: str(x), structure.positions.tolist())),
                    }
        if self._cs is None:
            self._cs = chemiscope.show([structure], mode="structure")
            handle.display(self._cs)
        else:
            self._cs.close()
            self._cs = chemiscope.show([structure], mode="structure")
            handle.update(self._cs)

In [ ]:
data_dump = WidgetDataDumper(prefix="ex_05")
display(data_dump)

### Fractional coordinates

*Fractional coordinate system* is the coordinate system over the *unit cell* basis. The *unit cell* basis is defined the *unit cell vectors* $\mathbf{a}$, $\mathbf{b}$ and $\mathbf{c}$. We can express any point $\mathbf{v}\in\mathbb{R}^3$ using a linear combination of the unit cell vectors $\mathbf{v}=n_1\mathbf{a}+n_2\mathbf{b}+n_3\mathbf{c}$, these points $(n_1,n_2,n_3)$ are the *fractional coordinates*.


#### Reading
- Crystal Structure Determination from Werner Massa: Chapter 2 - Crystal Lattices

<span style="color:blue"> **Warm up** Given the cubic unit cell as below with an edge lengths of 10 Angstrom. What are the fractional coordinates of all atoms in the unit cell? You can see the answer for an atom by clicking on it in the viewer and then get the corresponding information by clicking on the info icon below the viewer.</span>

In [ ]:
cell_length = 10
positions = np.array([[0,0,0], [0,1,0], [0,0,1], [0,1,1],
                      [0,1,0.5], [0.5,1,0], [0.5,0,1], [0,0.5,1],
                      [0.5,1,1], [0.5,0.5,1], [0.5,1,0.5],
                      [0.5,0,0], [0,0.5,0], [0,0,0.5], [0.5,0.5,0], [0.5,0,0.5], [0,0.5,0.5], [0.5,0.5,0.5]])
Ga_cuboid = ase.Atoms("18Ga", positions=(positions*a), cell=[a,a,a])

properties = {}
properties["index"] = {
                "target": "atom",
                "values": list(range(1, len(Ga_cuboid)+1)),
            }

properties["coordinates"] = {
                "target": "atom",
                "values": list(map(lambda x: str(x), (cell_length*positions).tolist())),
            }
properties["fractional coordinates"] = {
                "target": "atom",
                "values": list(map(lambda x: str(x), positions.tolist())),
            }

cs = chemiscope.show([Ga_cuboid], properties=properties, mode="structure", cutoff=8.5)
display(cs)

### Primitive cell

A primitive cell is a unit cell with minimal volume $|\det([\mathbf{a},\mathbf{b},\mathbf{c}])| = \mathbf{a}\cdot(\mathbf{b}\times\mathbf{c})$.

<span style="color:blue"> **Warm up** What is a primitive cell of the lattice above?</span>

In [ ]:
# TODO cleanly implemend widgets for solution for warmups
warmup01_wci = WidgetCodeInput(
        function_name="print_warmup01_solution_", 
        function_parameters="",
        docstring="",
        function_body=""
        )

layout = ipywidgets.widgets.Layout(width='auto', height='40px') #set width and height

warmup01_wt = ipywidgets.widgets.Text(
    value="It is the cubic cell defined by the unit cell vectors {(5,0,0), (0,5,0), (0,0,5)}.",
    placeholder='Type something',
    description='Solution:',
    display='flex',
    flex_flow='column',
    align_items='stretch',
    layout = layout,
    disabled=False
)

warmup01_wcc = WidgetCodeCheck(warmup01_wci, ref_values = 
                           { }, demo=WidgetPrint(warmup01_wt, WidgetParbox()))
display(warmup01_wcc)

### Supercell

A supercell is a multiple of a unit cell, for example the original presented unit cell $\{(10,0,0), (0,10,0), (0,0,10)\}$ is a supercell of the primitive cell we constructed above $\{(5,0,0), (0,5,0), (0,0,5)\}$. A super cell uses another smaller cell as building block. For the multiples only integer values are considered (no fractions of a cell). 

<span style="color:blue"> **Warm up** Try to construct the last cell as a supercell of the primitive one using the chemiscope interface: Click on the tab in the upper right corner indicated by three lines, then in the **Supercell** section expand the y- and z-directions to create a 122-supercell.</span>

In [ ]:
a = 5
positions = np.array([[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1],[1,1,1]])
Ga_cube = ase.Atoms("8Ga", positions=(positions*a), cell=[a,a,a])

properties = {}
properties["index"] = {
                "target": "atom",
                "values": list(range(1, len(Ga_cube)+1)),
            }

properties["coordinates"] = {
                "target": "atom",
                "values": list(map(lambda x: str(x), (a*positions).tolist())),
            }
properties["fractional coordinates"] = {
                "target": "atom",
                "values": list(map(lambda x: str(x), positions.tolist())),
            }

cs = chemiscope.show([Ga_cube], properties=properties, mode="structure", cutoff=8.5)
display(cs)

In [ ]:
# TODO make supercell as soons as default configurations are settable

<span style="color:blue"> **Warm up** In the above supercell some atoms are on top of each other. If you think about it, by having an atom at position (0,0,0) and (1,0,0) and by repeating the cell in the x-direction you get two atom at position (1,0,0). For a clearer visualization we have plotted so far the atoms in the neighbouring image too. But now we want to do only leave the correct number of atoms in the cell. For the periodic structure below try to create a 222- and 233-supercell.</span>

In [ ]:
a = 5
positions = np.array([[0,0,0]])
Ga_cube = ase.Atoms("1Ga", positions=(positions*a), cell=[a,a,a])

properties = {}
properties["index"] = {
                "target": "atom",
                "values": list(range(1, len(Ga_cube)+1)),
            }

properties["coordinates"] = {
                "target": "atom",
                "values": list(map(lambda x: str(x), (a*positions).tolist())),
            }
properties["fractional coordinates"] = {
                "target": "atom",
                "values": list(map(lambda x: str(x), positions.tolist())),
            }

cs = chemiscope.show([Ga_cube], properties=properties, mode="structure", cutoff=8.5)
display(cs)

<span style="color:blue"> **Warm up** Primitive cells are not unique. Can you construct a different primitive cell that constructs the same periodic structure? One trivial way to do this to translate the cell in a direction. Can you use rotation to create a new primitive cell from the original one. Define the new cell in the function below</span>

In [ ]:
# set upt the code widget window
warmup02_wci = WidgetCodeInput(
        function_name="construct_different_primitive_cell", 
        function_parameters="structure",
        docstring="""
Updates the cell of the structure and visualizes it

:param structure: the periodic structure with cell and position information. The cell is set to 
        
:return: Updated structure with a new cell
""",
        function_body="""

# Write your solution, and test creating 233-supercells from it

# You can change the unit cell from the structureplot_structure by accessing the member variable `cell` of structure using `structure.cell`.
# For example:
# a = [a_x,a_y,a_z]
#a = [1,2,3]
#b = [0,8,1]
#c = [4.5,3.1,5]
#structure.cell = [a,b,c]
return structure
"""
        )
warmup02_wcs = WidgetChemiscope(warmup02_wci, WidgetParbox(),
                               fixed_args={'structure': ase.Atoms("1Ga", positions=[[0,0,0]], cell=[5,5,5])})

warmup02_wcc = WidgetCodeCheck(warmup02_wci, ref_values = 
                           { }, demo=warmup02_wcs)
display(warmup02_wcc)

In [ ]:
wramup02_solution_wci = WidgetCodeInput(
        function_name="construct_different_primitive_cell", 
        function_parameters="structure",
        docstring="""
Updates the cell of the structure and visualizes it

:param structure: the periodic structure with cell and position information. The cell is set to 
        
:return: Updated structure with a new cell
""",
        function_body="""
# a = [a_x,a_y,a_z]
a = [-5,0,0]
b = [0,-5,0]
c = [0,0,-5]
structure.cell = [a,b,c]
return structure
"""
)

warmup02_solution_wcs = WidgetChemiscope(wramup02_solution_wci, WidgetParbox(),
                               fixed_args={'structure': ase.Atoms("1Ga", positions=[[0,0,0]], cell=[5,5,5])})

warmup02_solution_wcc = WidgetCodeCheck(wramup02_solution_wci, ref_values={},
                                        demo=warmup02_solution_wcs)
display(warmup02_solution_wcc)

### Read the chapters
Chapter 2 - Crystal Lattices
Section 3.4 - Lattice Planes and hk/-Indices

#### TODO Question should I include 8 types to make the question harder, by considering noncubic lattices  https://chemed.chem.purdue.edu/genchem/topicreview/bp/ch13/unitcell.php ?

<span style="color:blue"> **Exercise 01** What are the lattice types shown below? Possible types: face-centered cubic, body-centered cubic, primitive cubic, diamond cubic.</span>

In [ ]:
GaAs_diamond_cube = ase.io.read("sd_0311662.cif")
GaAs_diamond_cube.positions /= np.linalg.norm(GaAs_diamond_cube.cell, axis=0)[None,:]


In [ ]:
GaAs_diamond_cube.positions

In [ ]:
GaAs_diamond_cube.numbers

In [ ]:
GaAs_diamond_cube = ase.Atoms("Ga4As4", positions=[[0.  , 0.  , 0.  ],
                                                   [0.  , 0.5 , 0.5 ],
                                                   [0.5 , 0.  , 0.5 ],
                                                   [0.5 , 0.5 , 0.  ],
                                                   [0.25, 0.25, 0.25],
                                                   [0.75, 0.75, 0.25],
                                                   [0.75, 0.25, 0.75],
                                                   [0.25, 0.75, 0.75]],
                              cell=[1,1,1])


In [ ]:
# TODO rename such that answer cannot be inferred
cell_length = 5
positions = np.array([[0,0,0], [0,1,0], [0,0,1], [0,1,1],
                      [0,1,0.5], [0.5,1,0], [0.5,0,1], [0,0.5,1],
                      [0.5,1,1], [0.5,0.5,1], [0.5,1,0.5],
                      [0.5,0,0], [0,0.5,0], [0,0,0.5], [0.5,0.5,0], [0.5,0,0.5], [0,0.5,0.5], [0.5,0.5,0.5]])

#Ga_primitive_cube = ase.Atoms("8Ga", positions=[[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1],[1,1,1]], cell=[1,1,1])
Ga_primitive_cube = ase.Atoms("Ga", positions=[[0,0,0]], cell=[1,1,1])
Ga_primitive_cube.cell *= cell_length
Ga_primitive_cube.positions *= cell_length
#GaAs_fcc = ase.Atoms("Ga8As6", positions=[[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1],[1,1,1],
#                                          [0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5],[0.5,0.5,1],[0.5,1,0.5],[1,0.5,0.5]], cell=[1,1,1])
GaAs_fcc = ase.Atoms("GaAs3", positions=[[0,0,0],
                                          [0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]], cell=[1,1,1])
GaAs_fcc.cell *= cell_length
GaAs_fcc.positions *= cell_length

#GaAs_bcc = ase.Atoms("Ga8As", positions=[[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1],[1,1,1],[0.5,0.5,0.5]], cell=[1,1,1])
GaAs_bcc = ase.Atoms("GaAs", positions=[[0,0,0],[0.5,0.5,0.5]], cell=[1,1,1])
GaAs_bcc.cell *= cell_length
GaAs_bcc.positions *= cell_length

GaAs_diamond_cube = ase.Atoms("Ga4As4", positions=[[0.  , 0.  , 0.  ],
                                                   [0.  , 0.5 , 0.5 ],
                                                   [0.5 , 0.  , 0.5 ],
                                                   [0.5 , 0.5 , 0.  ],
                                                   [0.25, 0.25, 0.25],
                                                   [0.75, 0.75, 0.25],
                                                   [0.75, 0.25, 0.75],
                                                   [0.25, 0.75, 0.75]],
                              cell=[1,1,1])
GaAs_diamond_cube.positions /= np.linalg.norm(GaAs_diamond_cube.cell, axis=0)[None,:]
GaAs_diamond_cube.cell *= cell_length
GaAs_diamond_cube.positions *= cell_length

cs_primitive_cube  = chemiscope.show([Ga_primitive_cube], mode="structure")
cs_fcc = chemiscope.show([GaAs_fcc], mode="structure")
cs_bcc = chemiscope.show([GaAs_bcc], mode="structure")
cs_diamond_cube = chemiscope.show([GaAs_diamond_cube], mode="structure")

ipywidgets.GridBox(children=[cs_diamond_cube, cs_fcc, cs_primitive_cube, cs_bcc],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%')
       )

<span style="color:blue">  **Hint** Try to construct different supercells.</span>

In [ ]:
ex01_txt = Textarea("From left to right the lattice types are:", layout=Layout(width="100%"))
data_dump.register_field("ex01-answer", ex01_txt, "value")
display(ex01_txt)
#TODO save button

<span style="color:blue"> **Exercise 02** Construct the primitive cell for the second structure (without changing the atom's position, there is only one primitive cell)</span>

<span style="color:blue"> **Hint** Try to create the smallest cuboid containing all atoms in the unit cell (atoms from neighbouring images not included) and check if it defines the same periodic structure.</span>

In [ ]:
# set upt the code widget window
ex02_wci = WidgetCodeInput(
        function_name="construct_primitive_cell", 
        function_parameters="structure",
        docstring="""
Updates the cell of the structure and visualizes it

:param structure: the periodic structure with cell and position information. The cell is set to 
        
:return: Updated structure with a primitive cell
""",
        function_body="""

# Write your solution, and test creating 233-supercells from it

# You can change the unit cell from the structureplot_structure by accessing the member variable `cell` of structure using `structure.cell`.
# For example:
# a = [a_x,a_y,a_z]
#a = [1,2,3]
#b = [0,8,1]
#c = [4.5,3.1,5]
#structure.cell = [a,b,c]
return structure
"""
        )
ex02_wcs = WidgetChemiscope(ex02_wci, WidgetParbox(),
                               fixed_args={'structure': GaAs_fcc})

ex02_wcc = WidgetCodeCheck(ex02_wci, ref_values={}, demo=ex02_wcs)
display(ex02_wcc)

In [ ]:
# TODO move solution in folder and load it here with %load

In [ ]:
# set upt the code widget window
ex02_solution_wci = WidgetCodeInput(
        function_name="construct_primitive_cell", 
        function_parameters="structure",
        docstring="""
Updates the cell of the structure and visualizes it

:param structure: the periodic structure with cell and position information. The cell is set to 
        
:return: Updated structure with a primitive cell
""",
        function_body="""

# Write your solution, and test creating 233-supercells from it

# a = [a_x,a_y,a_z]
a = [0.5,0.5,0]
b = [0.5,0,0.5]
c = [0,0.5,0.5]
structure.cell = [a,b,c]

cell_length = structure.cell[0][0]
structure.cell *= cell_length
return structure
"""
        )
ex02_solution_wcs = WidgetChemiscope(ex02_solution_wci, WidgetParbox(),
                               fixed_args={'structure': ase.Atoms("GaAs3", positions=[[0,0,0],
                                          [2.5,2.5,0],[2.5,0,2.5],[0,2.5,2.5]], cell=[5,5,5])})

ex02_solution_wcc = WidgetCodeCheck(ex02_solution_wci, ref_values={}, demo=ex02_solution_wcs)
display(ex02_solution_wcc)

### Surface cuts

redifne Hower indices here

#### Reading
- Crystal Structure Determination from Werner Massa: Section 3.4 - Lattice Planes and hk/-Indices
- Interfaces in Materials from James M. Howe: Section 4.3 - Surface crystallography

In [ ]:
# cell length
a = GaAs_fcc.cell[0][0]

# translation look at the cell below
GaAs_fcc_111_surface = copy.deepcopy(GaAs_fcc)
GaAs_fcc_111_surface.positions -= [0,0,a]
GaAs_fcc_111_surface.cell[2] = [0,0,-a]

def add_periodic_image(structure, direction):
    cell_length = GaAs_fcc.cell[0][0]
    periodic_image = copy.deepcopy(structure)
    periodic_image.positions += np.array(direction)*cell_length
    structure += periodic_image

add_periodic_image(GaAs_fcc_111_surface, [1,0,0])
add_periodic_image(GaAs_fcc_111_surface, [0,1,0])
add_periodic_image(GaAs_fcc_111_surface, [0,0,1])

In [ ]:
import copy

cell_length = GaAs_fcc.cell[0][0]

# translation look at the cell below
GaAs_fcc_00m1_translated = copy.deepcopy(GaAs_fcc)
GaAs_fcc_00m1_translated.positions -= [0,0,cell_length]
GaAs_fcc_00m1_translated.cell[2] = [0,0,-cell_length]
GaAs_fcc_111_surface = copy.deepcopy(GaAs_fcc_00m1_translated)


def add_periodic_image(structure, unit_cell_structure, direction):
    cell_length = unit_cell_structure.cell[0][0]
    periodic_image = copy.deepcopy(unit_cell_structure)
    periodic_image.positions += np.array(direction)*cell_length
    structure += periodic_image

add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [1,0,0])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [0,1,0])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [0,0,1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [1,1,0])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [1,0,1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [0,1,1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [1,1,1])

add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [-1,0,0])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [0,-1,0])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [0,0,-1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [-1,-1,0])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [-1,0,-1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [0,-1,-1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [-1,-1,-1])

add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [-1,1,0])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [1,-1,0])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [-1,0,1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [1,0,-1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [0,-1,1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [0,1,-1])

add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [-1,1,1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [1,-1,1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [1,-1,-1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [-1,-1,1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [-1,1,-1])
add_periodic_image(GaAs_fcc_111_surface, GaAs_fcc, [1,-1,-1])

In [ ]:
normal_vec_surface_1 = np.array([1,1,1])
idx_lower = np.where((GaAs_fcc_111_surface.positions @ normal_vec_surface_1) >= 0)[0]

trirectangular_tetrahedron_height = np.sqrt((cell_length/2)**2 + cell_length/2**2)
unit_cell_diagonal_length = cell_length * np.sqrt(3)
a_length = unit_cell_diagonal_length - 2*trirectangular_tetrahedron_height
a_unit_cell_vector = a_length*np.array([1,1,1])/np.linalg.norm(np.array([1,1,1]))

idx_upper = np.where(((GaAs_fcc_111_surface.positions-a_length) @ normal_vec_surface_1) < 0)[0]
GaAs_fcc_111_surface_ = ase.Atoms([GaAs_fcc_111_surface[i] for i in set(idx_lower).intersection(set(idx_upper))])
GaAs_fcc_111_surface_.cell = GaAs_fcc_111_surface.cell

<span style="color:blue"> **Exercise 03** What is the unit cell for a (111)-surface-cut on the second structure. You can have a look at the surface cut on a supercell below. Do not use the supercell function on that periodic structure as the unit cell for the surface is not correctly defined. If you look at the (x,y,z) vectors, you can see that we applied the surface cut on the periodic image in the (0,0,-1)-direction. This allows to define the unit cell for the (111)-surface-cut from the (0,0,0) origin.</span>

<span style="color:blue"> **Hint** The direction of one unit cell vector is given by the surface cut. The norm of this unit cell vector can be computed by subtracting the height of the two trirectangular tetrahedron which are cutted away. The other two unit cell vectors have to be on the surface.</span>

<img src="figures/111-surface-cut.png" width="300" height="150" />
(111) surface cut
Source: https://en.wikipedia.org/wiki/Miller_index

In [ ]:
cs = chemiscope.show([GaAs_fcc_111_surface_], mode="structure")
display(cs)

In [ ]:
# set upt the code widget window
ex03_wci = WidgetCodeInput(
        function_name="construct_111_surface_unit_cell", 
        function_parameters="structure",
        docstring="""
Updates the cell of the structure and visualizes it

:param structure: the periodic structure with cell and position information. The cell is set to 
        
:return: Updated structure with for surface cut (111)
""",
        function_body="""

# some math functionalities
from numpy import sqrt
from numpy.linalg import norm

# Write your solution here

return structure
"""
        )
ex03_wcs = WidgetChemiscope(ex03_wci, WidgetParbox(),
                               fixed_args={'structure': copy.deepcopy(GaAs_fcc_111_surface_)})

ex03_wcc = WidgetCodeCheck(ex03_wci, ref_values={}, demo=ex03_wcs)
display(ex03_wcc)

In [ ]:
# set upt the code widget window
ex03_solution_wci = WidgetCodeInput(
        function_name="construct_111_surface_unit_cell", 
        function_parameters="structure",
        docstring="""
Updates the cell of the structure and visualizes it

:param structure: the periodic structure with cell and position information. The cell is set to 
        
:return: structure with updated unit cell for surface cut (111)
""",
        function_body="""
from numpy import sqrt
from numpy.linalg import norm

cell_length = structure.cell[0][0]
structure.cell = [[1,1,1], [1,-1,0], [1,1,-2]]
## compute length of unit cell vector a
# formula for height of trirectangular tetrahedron (two times pythagoras + height of triangles)
trirectangular_tetrahedron_height = sqrt((cell_length/2)**2 + cell_length/2**2)
unit_cell_diagonal_length = cell_length * sqrt(3)
a_length = unit_cell_diagonal_length - 2*trirectangular_tetrahedron_height
structure.cell[0] /= norm(structure.cell[0])
structure.cell[0] *= a_length

## compute length of unit cell vector b and c
structure.cell[1] *= cell_length
structure.cell[2] *= cell_length
return structure
"""
        )
ex03_solution_wcs = WidgetChemiscope(ex03_solution_wci, WidgetParbox(),
                               fixed_args={'structure': copy.deepcopy(GaAs_fcc_111_surface_)})

ex03_solution_wcc = WidgetCodeCheck(ex03_solution_wci, ref_values={}, demo=ex03_solution_wcs)
display(ex03_solution_wcc)
